# WasteNet

## Setup

In [ ]:
# Colab
!rm -r smart-cities-drl
!git clone https://github.com/eescriba/smart-cities-drl
!cd smart-cities-drl/ && pip install -e .

import sys
sys.path.insert(0,'./smart-cities-drl/src/')

# Local
# !pip install -e ..
# import sys
# sys.path.insert(0,'../src/')

In [2]:
import json
import shutil
import random

import gym
import ray
from ray.tune import run, choice
from core.rl import PPOAgent
from core.pbt import PbtOptimizer
from wastenet.env import WasteNetEnv

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


Instructions for updating:
experimental_compile is deprecated, use jit_compile instead


## Proximal Policy Optimization (PPO)

### Tune hyperparameters

In [3]:
ppo = PPOAgent("wastenet_ppo_tune", WasteNetEnv, {})

2021-06-25 06:54:32,717	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265
2021-06-25 06:54:35,580	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-06-25 06:54:35,582	INFO trainer.py:698 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=256) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprecated and will be removed in a future version.
(pid=256) Instructions for updating:
(pid=256) experimental_compile is deprecated, use jit_compile instead
(pid=257) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprec

In [4]:
tune_config = {
    "env": WasteNetEnv,
    "seed": 123,
    "num_gpus": 1,
    "num_workers": 1,
    "observation_filter": "MeanStdFilter",
    "model": {
        "free_log_std": True
    },
    "lambda": 0.95,
    "clip_param": 0.3,
    "lr": 5e-5,
    "num_sgd_iter": choice([10, 20, 30]),
    "sgd_minibatch_size": choice([128, 256, 512]),
    "train_batch_size": choice([8000, 16000, 32000])
}
stop_criteria = {
    "timesteps_total": 2000000
}
hyperparam_mutations={
    "lambda": lambda: random.uniform(0.7, 1.0),
    "clip_param": lambda: random.uniform(0.1, 0.5),
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "num_sgd_iter": lambda: random.randint(1, 30),
    "sgd_minibatch_size": lambda: random.randint(128, 16384),
    "train_batch_size": lambda: random.randint(2000, 160000),
}
pbt = PbtOptimizer(hyperparam_mutations)

In [ ]:
ppo.restart()
analysis = ppo.tune(tune_config, stop_criteria, scheduler=pbt.scheduler)
best_config =  analysis.get_best_config(metric="episode_reward_mean", mode="max")
print("Best hyperparameters found: ", best_config)

2021-06-25 06:57:02,354	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc,num_sgd_iter,sgd_minibatch_size,train_batch_size
PPO_WasteNetEnv_8c324_00000,PENDING,,20,512,20000
PPO_WasteNetEnv_8c324_00001,PENDING,,20,512,20000
PPO_WasteNetEnv_8c324_00002,PENDING,,20,2048,10000
PPO_WasteNetEnv_8c324_00003,PENDING,,10,2048,10000
PPO_WasteNetEnv_8c324_00004,PENDING,,10,128,10000
PPO_WasteNetEnv_8c324_00005,PENDING,,10,512,20000
PPO_WasteNetEnv_8c324_00006,PENDING,,30,128,20000
PPO_WasteNetEnv_8c324_00007,PENDING,,10,128,20000


(pid=463) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprecated and will be removed in a future version.
(pid=463) Instructions for updating:
(pid=463) experimental_compile is deprecated, use jit_compile instead
(pid=463) 2021-06-25 06:57:07,532	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=463) 2021-06-25 06:57:07,532	INFO trainer.py:698 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc,num_sgd_iter,sgd_minibatch_size,train_batch_size
PPO_WasteNetEnv_8c324_00000,RUNNING,,20,512,20000
PPO_WasteNetEnv_8c324_00001,PENDING,,20,512,20000
PPO_WasteNetEnv_8c324_00002,PENDING,,20,2048,10000
PPO_WasteNetEnv_8c324_00003,PENDING,,10,2048,10000
PPO_WasteNetEnv_8c324_00004,PENDING,,10,128,10000
PPO_WasteNetEnv_8c324_00005,PENDING,,10,512,20000
PPO_WasteNetEnv_8c324_00006,PENDING,,30,128,20000
PPO_WasteNetEnv_8c324_00007,PENDING,,10,128,20000


(pid=464) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprecated and will be removed in a future version.
(pid=464) Instructions for updating:
(pid=464) experimental_compile is deprecated, use jit_compile instead
(pid=464) WARNING:tensorflow:
(pid=464) The following Variables were used a Lambda layer's call (lambda), but
(pid=464) are not present in its tracked objects:
(pid=464)   <tf.Variable 'default_policy/log_std:0' shape=(1,) dtype=float32>
(pid=464) It is possible that this is intended behavior, but it is more likely
(pid=464) an omission. This is a strong indication that this layer should be
(pid=464) formulated as a subclassed Layer rather than a Lambda layer.
(pid=463) WARNING:tensorflow:
(pid=463) The following Variables were used a Lambda layer's call (lambda), but
(pid=463) are not present in its tracked

...

Trial name,status,loc,num_sgd_iter,sgd_minibatch_size,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_WasteNetEnv_8c324_00000,PAUSED,,20,512,20000,18,408.662,360000,-22.56,29,-106,300
PPO_WasteNetEnv_8c324_00001,PAUSED,,20,512,20000,18,407.923,360000,-21.23,27,-105,300
PPO_WasteNetEnv_8c324_00003,PAUSED,,6,122,14400,22,397.358,400860,-89.58,-29,-175,300
PPO_WasteNetEnv_8c324_00004,PAUSED,,10,128,10000,30,384.725,300000,-22.91,34,-162,300
PPO_WasteNetEnv_8c324_00005,PAUSED,,12,153,12000,26,372.808,292000,-70.83,1,-161,300
PPO_WasteNetEnv_8c324_00006,PAUSED,,8,153,12000,29,383.745,308000,-16.52,38,-85,300
PPO_WasteNetEnv_8c324_00007,PAUSED,,10,128,20000,15,381.21,300000,-28.27,27,-112,300
PPO_WasteNetEnv_8c324_00002,PENDING,,20,2048,10000,36,365.104,360000,-55.74,7,-137,300


(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=3593) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprecated and will be removed in a future version.
(pid=3593) Instructions for u

Trial name,status,loc,num_sgd_iter,sgd_minibatch_size,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_WasteNetEnv_8c324_00002,RUNNING,,20,2048,10000,36,365.104,360000,-55.74,7,-137,300
PPO_WasteNetEnv_8c324_00000,PAUSED,,20,512,20000,18,408.662,360000,-22.56,29,-106,300
PPO_WasteNetEnv_8c324_00001,PAUSED,,20,512,20000,18,407.923,360000,-21.23,27,-105,300
PPO_WasteNetEnv_8c324_00003,PAUSED,,6,122,14400,22,397.358,400860,-89.58,-29,-175,300
PPO_WasteNetEnv_8c324_00004,PAUSED,,10,128,10000,30,384.725,300000,-22.91,34,-162,300
PPO_WasteNetEnv_8c324_00006,PAUSED,,8,153,12000,29,383.745,308000,-16.52,38,-85,300
PPO_WasteNetEnv_8c324_00007,PAUSED,,10,128,20000,15,381.21,300000,-28.27,27,-112,300
PPO_WasteNetEnv_8c324_00005,PENDING,,12,153,12000,26,372.808,292000,-70.83,1,-161,300


(pid=3590) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprecated and will be removed in a future version.
(pid=3590) Instructions for updating:
(pid=3590) experimental_compile is deprecated, use jit_compile instead
(pid=3590) WARNING:tensorflow:
(pid=3590) The following Variables were used a Lambda layer's call (lambda), but
(pid=3590) are not present in its tracked objects:
(pid=3590)   <tf.Variable 'default_policy/log_std:0' shape=(1,) dtype=float32>
(pid=3590) It is possible that this is intended behavior, but it is more likely
(pid=3590) an omission. This is a strong indication that this layer should be
(pid=3590) formulated as a subclassed Layer rather than a Lambda layer.
(pid=3593) WARNING:tensorflow:
(pid=3593) The following Variables were used a Lambda layer's call (lambda), but
(pid=3593) are not present i

Trial name,status,loc,num_sgd_iter,sgd_minibatch_size,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_WasteNetEnv_8c324_00002,RUNNING,,20,2048,10000,36,365.104,360000,-55.74,7,-137,300
PPO_WasteNetEnv_8c324_00000,PAUSED,,20,512,20000,18,408.662,360000,-22.56,29,-106,300
PPO_WasteNetEnv_8c324_00001,PAUSED,,20,512,20000,18,407.923,360000,-21.23,27,-105,300
PPO_WasteNetEnv_8c324_00003,PAUSED,,6,122,14400,22,397.358,400860,-89.58,-29,-175,300
PPO_WasteNetEnv_8c324_00004,PAUSED,,10,128,10000,30,384.725,300000,-22.91,34,-162,300
PPO_WasteNetEnv_8c324_00006,PAUSED,,8,153,12000,29,383.745,308000,-16.52,38,-85,300
PPO_WasteNetEnv_8c324_00007,PAUSED,,10,128,20000,15,381.21,300000,-28.27,27,-112,300
PPO_WasteNetEnv_8c324_00005,PENDING,,12,153,12000,26,372.808,292000,-70.83,1,-161,300


In [ ]:
best_config = {
    "observation_filter": "MeanStdFilter",
    "model": {"free_log_std": True},
    "num_sgd_iter": 10,
    "sgd_minibatch_size": 128,
    "lambda": 0.731396,
    "clip_param": 0.317651,
    "lr": 5e-05,
    "train_batch_size": 18812,
}

### Training

In [ ]:
ppo = PPOAgent("wastenet_ppo_train", best_config, WasteNetEnv, {})
ppo.train(num_iter=200)

In [ ]:
policy = ppo.agent.get_policy()
model = policy.model
print(model.base_model.summary())

### Testing

In [ ]:
# ppo = PPOAgent("wastenet_ppo_test", best_config, WasteNetEnv, {})
# ppo.load("checkpoints/checkpoint-best")
ppo.test(num_episodes=1000)

### Visualization

In [ ]:
!zip -r /content/ray_results.zip /content/ray_results

In [ ]:
%load_ext tensorboard 
%tensorboard --logdir="/content/ray_results/"